# What Should I Make for Dinner - a mini-app!
## Description:
This mini app is useful for when you need to use up food that's going bad, but you're out of ideas on what to make! Based on a list of ingredients that you're wanting to utilize and any dietary restrictions you may have, it will output a list of meal ideas!

---

## Dependencies to install:
`pip install python-dotenv`

`pip install -q -U google-generativeai`

---

In [4]:
import os
from dotenv import load_dotenv
import requests
import itertools
import google.generativeai as genai
from IPython.display import display, Markdown

load_dotenv()
SPOONACULAR_API_KEY = os.getenv('SPOONACULAR_API_KEY')

### To begin, you enter a list of ingredients you have in your fridge that you're wanting to use up before they go bad as well as any dietary restrictions.

In [5]:
ingredients: str = input('Enter ingredients here')
while not ingredients:
    print('Please enter a valid list of ingredients')
    ingredients: str = input("Enter ingredients here")
dietary_restrictions: str = input(
    'What are your dietary restrictions? If you don\'t have any type \"none\"')
if not dietary_restrictions or dietary_restrictions == None:
    dietary_restrictions = 'There are no dietary restrictions.'

### We then call the spoonacular API to find recipe ideas with the inputted ingredients. Then we use the Gemini SDK to ensure only recipe ideas with ingredients that follow the provided dietary restrictions are given.

In [ ]:
spoonacular_uri = 'https://api.spoonacular.com/recipes/findByIngredients'
params = {
    'ingredients': ingredients,
    'number': 5,
    'ranking': 1,
    'ignorePantry': True
}
headers = {
    'Content-Type': 'application/json',
    'x-api-key': os.environ['SPOONACULAR_API_KEY']
}

response = requests.get(url=spoonacular_uri,
                        params=params, headers=headers)

if response.status_code != 200:
    print('There was an error with the Spoonacular API integration. Please try again.')
elif not response.json():
    print('No recipes with the given ingredient(s) were found.')
else:
    recipes_decoded = {}

    for recipe in response.json():
        all_ingredients = []
        for i in itertools.chain(recipe['missedIngredients'], recipe['usedIngredients']):
            all_ingredients.append(i['name'])
        recipes_decoded[recipe['title']] = all_ingredients

    genai.configure(api_key=os.environ["GOOGLE_AI_API_KEY"])

    model = genai.GenerativeModel("gemini-1.5-flash")

    try:
        gemini_response = model.generate_content(
            f'Based on the list of recipes and subsequent ingredients provided, return the recipes' +
            f'that follow the following dietary restriction(s): {dietary_restrictions}.' +
            f'If there are ingredients in the recipes that don\'t follow the dietary restrictions, substitute them. The recipes are: {recipes_decoded}.' +
            f"Include these ingredients: {ingredients} and the dietary restrictions (if they have them) in the top heading"
        )
    except:
        print('There was an error with the Gemini Integration. Please try again.')
        exit()

    display(Markdown(gemini_response.text))